In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
np.random.seed(666)

#physical_devices = tf.config.list_physical_devices('GPU')

#tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
file= open('/Users/eiser/Desktop/oasis_cross-sectional_labels.txt','r')

ligne= file.readlines()

file.close()

# Séparation des images en ensembles d'entraînement, validation et test

In [ ]:
noms = []
alzheimer = []
homme_alz=[]
homme_saint=[]
femme_alz=[]
femme_saint=[]

for i in range (1,len(ligne)):
    li=str(ligne[i]).rstrip('\n')
    liste=li.split(',')
    #print(liste)
    if liste[1]== 'F':
        
        if liste[7] != "":
            noms.append(liste[0])
            if float(liste[7]) <=0.5:
                femme_saint.append(liste[0])
            else:
                femme_alz.append(liste[0])
            

print(len(femme_alz))
print(len(femme_saint))

In [ ]:
import os, shutil 

src='/Users/eiser/Desktop/'+femme_alz[1]+'.jpg'
os.path.isfile(src)==True

In [ ]:
femme_alz_train=femme_alz[:14]
femme_alz_test=femme_alz[14:]
femme_saint_train=femme_saint[:100]
femme_saint_test=femme_saint[100:]

In [ ]:
src1= '/Users/eiser/Desktop/oasis_brain_no_nn_slices/X/Alzheimer/'
dst1='/Users/eiser/Desktop/oasis/femme/train/snt/'
for i in femme_saint_train:
    
    
    
    fname = i + '_mpr_nn_anon_111_t88_masked_gfcalzheimer.x77.jpg'
    src=src1+fname
    dst=dst1+ fname
    if os.path.isfile(src)==True:
        shutil.copyfile(src, dst)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
image_size = (176, 208)
batch_size = 32
train_dir='/Users/eiser/Desktop/oasis/femme/train/'
training_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_size),
    batch_size=batch_size,
    class_mode='binary'
)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_dir='/Users/eiser/Desktop/oasis/femme/test/'
test_set = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

## Définition du modèle et entraînement

In [ ]:
from keras import layers
from keras import models

model =models.Sequential()
model.add(layers.Conv2D(32, (3,3),activation='relu', input_shape=(176,208,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation ='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation ='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation ='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
from keras.applications import VGG16
conv_base = VGG16(weights='imagenet',include_top=False, input_shape=(176,208,3))

In [ ]:
from keras import models
from keras import layers



model=models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(95, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
from keras import optimizers 
import keras

model.compile(loss='binary_crossentropy',optimizer=optimizers.RMSprop(lr=1e-4),metrics=['accuracy'])

## Traçage des graphiques d'entraînement et ROC

In [ ]:
history= model.fit_generator(training_set,steps_per_epoch=20, shuffle=True, epochs=5)

In [ ]:
test_loss,  test_acc = model.evaluate(test_set)

In [ ]:
test_loss, test_tp, test_fp, test_tn, test_fn, test_acc, test_prec, test_recall, test_auc = model.evaluate(test_set)

In [ ]:
from keras.preprocessing import image

test_image = image.load_img('/Users/eiser/Desktop/oasis/femme/test/snt/OAS1_0352_MR1_mpr_nn_anon_111_t88_masked_gfcalzheimer.x77.jpg', target_size = (176, 208,3))
test_image = image.img_to_array(test_image)/255.
test_image = np.expand_dims(test_image, axis=0)

prediction = model.predict_on_batch(test_image)
print(prediction)

In [ ]:
import matplotlib.pyplot as plt

acc= history.history['acc']
#val_acc = history.history['val_accuracy']
loss= history.history['loss']
#val_loss= history.history['val_loss']

epochs= range(1, len(acc)+1)

plt.plot(epochs, acc, 'bo', label='entrainement')
#plt.plot(epochs, val_acc,'b', label='validation')
plt.title('Exactitude pendant l\'entrainement et la validation')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='entrainement')
#plt.plot(epochs, val_loss, 'b', label='validation')
plt.title('Perte pendant l\'entrainement et la validation')
plt.legend()
plt.show()

In [ ]:
test_acc

In [ ]:
import numpy as np
y_true_labels = test_set.classes
true_labels = []

for label in y_true_labels:
    true_labels.append(np.array([np.float32(label)]))

In [ ]:
preds = model.predict(test_set)

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
fpr, tpr, _ = roc_curve(true_labels, preds.tolist())
auc = auc(fpr, tpr)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='Keras (area = {:.3f})'.format(auc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

In [ ]:
test_loss, test_acc= model.evaluate(test_set)

## Gradcam

In [ ]:
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

In [ ]:
img_path='/Users/eiser/Desktop/oasis/femme/test/snt/OAS1_0352_MR1_mpr_nn_anon_111_t88_masked_gfcalzheimer.x77.jpg'

In [ ]:
img_path='/Users/eiser/Desktop/BIO/test/0/BioID_0014.pgm.jpg'

In [ ]:
from keras.preprocessing import image
img= image.load_img(img_path, target_size=(176,208,3))

x= image.img_to_array(img)

x= np.expand_dims(x, axis=0)

x= preprocess_input(x)

In [ ]:
x.shape

In [ ]:
preds=model.predict_proba(x)

In [ ]:
preds[0][0]

In [ ]:
decode_predictions(preds,top=1)[0]

In [ ]:
vggface.summary()

In [ ]:

from keras_vggface.vggface import VGGFace

# Based on VGG16 architecture -> old paper(2015)
vggface = VGGFace(model='vgg16') # or VGGFace() as default
from keras import backend as K
sortie= vggface.output[:, 1]

last_conv_layer = vggface.get_layer('conv5_1')

grads = K.gradients(sortie, last_conv_layer.output)[0]

pooled_grads = K.mean(grads, axis=(0,1,2))

iterate = K.function([vggface.input], [pooled_grads, last_conv_layer.output[0]])

pooled_grads_value, conv_layer_output_value = iterate([x])

for i in range(512):
    conv_layer_output_value[:,:,i] *= pooled_grads_value[i]
    
    
heatmap = np.mean(conv_layer_output_value, axis=-1)



In [ ]:
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
import matplotlib.pyplot as plt
plt.matshow(heatmap)


In [ ]:
import cv2

img = cv2.imread(img_path)

heatmap= cv2.resize(heatmap, (img.shape[1], img.shape[0]))

heatmap = np.uint8(255 * heatmap)

heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

superimposed_img = heatmap * 0.4 + img


cv2.imwrite('/Users/eiser/Desktop/cerveau_cam.jpg', superimposed_img)